In [1]:
import numpy as np
import matplotlib.pyplot as plt
import collections

In [2]:
# service handle
path = '/home/rei/reigit/service_request.txt'
f = open(path)
f.close()
with open(path) as f:
    l = f.readlines()
sh=[]
for i in l:
    sh.append(i.split('0x')[1].split(',')[0])
sh2=[]
for i in sh:
    sh2.append(str(int(i,16)))
s_counter=collections.Counter(sh2)  #how many time did they run?
service_counter=dict(s_counter)

#connect handle to service name
path = '/home/rei/reigit/avpdemo.txt'
f = open(path)
f.close()
with open(path) as f:
    l = f.readlines()
#client handle
S = l.index('Services:\n')
E = l.index('Clients:\n')
service_sn=[]
for i in l[S+3:E-1]:
    for j in service_counter.keys():
         if j == i.split()[0]:
             service_sn.append(j+':'+i.split()[4]+'--'+str(service_counter.get(str(j))))
for i in service_sn:
    print(i)

94683481026128:/planning/estimate_collision--600
94864481839632:/had_maps/HAD_Map_Service--6
94832173285216:/planning/plan_lane_trajectory/_action/send_goal--1
94832173285232:/planning/plan_lane_trajectory/_action/get_result--1
94322865568000:/planning/plan_parking_trajectory/_action/send_goal--1
94322865568016:/planning/plan_parking_trajectory/_action/get_result--1


In [3]:
# client handle
path = '/home/rei/reigit/client_response.txt'
f = open(path)
f.close()
with open(path) as f:
    l = f.readlines()
ch=[]
for i in l:
    ch.append(i.split('0x')[1].split(',')[0])
ch2=[]
for i in ch:
    ch2.append(str(int(i,16)))
c_counter=collections.Counter(ch2)  #how many time did they run?
client_counter=dict(c_counter)

#connect handle to service name
path = '/home/rei/reigit/avpdemo.txt'
f = open(path)
f.close()
with open(path) as f:
    l = f.readlines()
#client handle
S = l.index('Clients:\n')
E = l.index('Timers:\n')
client_sn=[]
for i in l[S+3:E-1]:
    for j in client_counter.keys():
         if j == i.split()[0]:
             client_sn.append(j+':'+i.split()[4]+'--'+str(client_counter.get(str(j))))
for i in client_sn:
    print(i)

94729055262288:/had_maps/HAD_Map_Service--1
94387741707344:/had_maps/HAD_Map_Service--1
94322865305360:/had_maps/HAD_Map_Service--1
94832173006016:/had_maps/HAD_Map_Service--1
139961240727904:/had_maps/HAD_Map_Service--1
94656882958896:/planning/plan_lane_trajectory/_action/send_goal--1
94656882958912:/planning/plan_lane_trajectory/_action/get_result--1
94656883378480:/planning/plan_parking_trajectory/_action/send_goal--1
94656882796928:/had_maps/HAD_Map_Service--1
94656882801504:/planning/estimate_collision--600


In [9]:
# find service pairs
pairs=[]
for i in service_sn:
    for j in client_sn:
        if i.split(':')[1]==j.split(':')[1]:
            ser=hex(int(i.split(':')[0]))
            cli=hex(int(j.split(':')[0]))
            pairs.append(ser+':'+cli+':'+j.split(':')[1])
print('service:client:service name')
print(pairs)

# send goal
sendh = [line for line in pairs if 'send_goal' in line]
send_handle=[]

# get result
geth = [line for line in pairs if 'get_result' in line]
get_handle=[]

# does they has the same service name?
for i in sendh:
    for j in geth:
        if i.split(':')[2].split('send_goal')[0] == j.split(':')[2].split('get_result')[0]:
            send_handle.append(i.split(':')[0].upper().replace('X', 'x'))
            get_handle.append(j.split(':')[1].upper().replace('X', 'x'))

print(sendh,send_handle)
print(geth,get_handle)

service:client:service name
['0x561d3740ae50:0x561705dffb60:/planning/estimate_collision--600', '0x563fd6007f60:0x561705e26230:/planning/plan_lane_trajectory/_action/send_goal--1', '0x563fd6007f70:0x561705e26240:/planning/plan_lane_trajectory/_action/get_result--1', '0x55c940e57900:0x561705e8c930:/planning/plan_parking_trajectory/_action/send_goal--1']
['0x563fd6007f60:0x561705e26230:/planning/plan_lane_trajectory/_action/send_goal--1', '0x55c940e57900:0x561705e8c930:/planning/plan_parking_trajectory/_action/send_goal--1'] ['0x563FD6007F60']
['0x563fd6007f70:0x561705e26240:/planning/plan_lane_trajectory/_action/get_result--1'] ['0x561705E26240']


In [10]:
# timestamp of send goal
def send_goal(send_handle):
    path = '/home/rei/reigit/service_request.txt'
    f = open(path)
    f.close()
    with open(path) as f:
        l = f.readlines()
    send_time=[]
    for i in l:
        service = i.split(",")[4].split(" = ")[1]
        if (service == send_handle):
            send_time.append(i.split(",")[5].split(" = ")[1])
    return send_time

for i in send_handle:
    send_timestamp=send_goal(i)

In [11]:
# timestamp of get result
def get_result(get_handle):
    path = '/home/rei/reigit/client_response.txt'
    f = open(path)
    f.close()
    with open(path) as f:
        l = f.readlines()
    get_time=[]
    for i in l:
        client = i.split(",")[4].split(" = ")[1]
        if (client == get_handle):
            get_time.append(i.split(",")[6].split(" = ")[1].split()[0])
    return get_time

for i in get_handle:
    get_timestamp=get_result(i)

In [12]:
# action latency
action_latency = [int(get_timestamp[i])-int(send_timestamp[i]) for i in range(min(len(get_timestamp),len(send_timestamp)))]
for i in action_latency:
    i=i/1000
    print(str(i)+' us')
print(send_timestamp,get_timestamp)

230645.562 us
['1639903511284341853'] ['1639903511514987415']
